# Import

In [1]:
import re
import pandas        as pd
import numpy         as np

# Load DataSet

In [2]:
df = pd.read_csv('../dataset/list_auto.csv', encoding='utf-8', sep=';')


In [3]:
df.head()

,id_ads,manufacturer_by,model,version,km,year,price,seller,endereco,scrapy_datetime,address
0,8c9aef9c-1513-41ab-8db4-10686decdfb5,bmw,116,d 5p Msport auto,146000,06-2019,16500,Bryan Auto Srl,BRYAN AUTO • IT-13100 Vercelli,2024-11-22,NaN
1,78b02ce1-2c8f-4caf-88fc-7d59b6e42cda,bmw,x1,xdrive18d Business auto my18,100000,09-2018,18999,Bryan Auto Srl,BRYAN AUTO • IT-13100 Vercelli,2024-11-22,NaN
2,c3eb9e5d-f4e3-4864-a283-32063782cd8b,bmw,318,d Touring MHEV Business Advantage 150CV auto,140000,11-2020,19490,Bryan Auto Srl,BRYAN AUTO • IT-13100 Vercelli,2024-11-22,NaN
3,a564e061-226c-464f-bf1a-b178e7b929dc,bmw,116,116d 5p. Msport,53050,02-2019,19900,Bryan Auto Srl,BRYAN AUTO • IT-13100 Vercelli,2024-11-22,NaN
4,6e6ae0b6-d199-4d2e-96ec-d525bfb48e1e,bmw,730,d xDrive Eccelsa,226135,02-2018,25900,Bryan Auto Srl,BRYAN AUTO • IT-13100 Vercelli,2024-11-22,NaN


# Using REGEX langague to clean and capture data-info

In [4]:
df_clean = df.copy()

In [7]:
# "Extracting the postal code from the vehicle's country of origin. 
# Applying Regex for data cleaning."
df_clean['zip_code'] = df_clean['endereco'].apply(lambda x: int(re.search('\d+', x).group(0) if pd.notnull( x ) else x ))

<>:3: SyntaxWarning: invalid escape sequence '\d'
<>:3: SyntaxWarning: invalid escape sequence '\d'
C:\Users\bruno\AppData\Local\Temp\ipykernel_1816\1301195681.py:3: SyntaxWarning: invalid escape sequence '\d'
  df_clean['zip_code'] = df_clean['endereco'].apply(lambda x: int(re.search('\d+', x).group(0) if pd.notnull( x ) else x ))


ValueError: cannot convert float NaN to integer

In [10]:
# Model
df_clean['model'] = 'DEFENDER' + ' ' + df_clean['version'].apply(lambda x: ' '.join(re.findall(r'\b(88|90|110|130)\b', x)) if re.findall(r'\d+', x) else None)

In [11]:
# Version Type
df_clean['engine'] = df_clean['version'].apply(lambda x: ' '.join(re.findall(r'[A-Za-z]+\d+', x)) if re.findall(r'\d+', x) else None)
df_clean['engine_version'] = df_clean['version'].apply(lambda x: ' '.join(re.findall(r'(AWD|SW|SE|HSE|MHEV|HT)', x)) if re.findall(r'\d+', x) else None)
df_clean['cc_engine'] = df_clean['version'].apply(lambda x: ' '.join(re.findall(r'\b\d\.\d', x)) if re.findall(r'\d+', x) else None)

# Replacing model version values.
df_clean['version'] = df_clean.apply(lambda row: row['engine'] + ' ' + row['engine_version'] + ' ' + row['cc_engine'] if row['engine'] is not None and row['engine_version'] is not None and row['cc_engine'] is not None else None, axis=1)


In [12]:
#Country
df_clean['country'] = df_clean['endereco'].apply(lambda x: str(re.search(r"([A-Z]{2})", x).group(0) if pd.notnull( x ) else x ))

In [13]:
# DataTime cleaning

df_clean['year'] = df_clean['year'].str.replace('-', '/')


for i in range(len(df_clean)):
    if df_clean['year'][i] == 'new' or df_clean['year'][i] == 'unknown' or df_clean['year'][i] == 'None':
        df_clean.at[i, 'year'] = '01/2023'


df_clean['year'] = pd.to_datetime(df_clean['year'], format='%m/%Y')

df_clean['year'] = df_clean['year'].dt.year


## DataFrame Final

In [14]:
df_final = pd.DataFrame(df_clean[['id_ads', 'manufacturer_by', 'model', 'version', 'km', 'year',
                                  'price', 'seller', 'endereco', 'scrapy_datetime', 'zip_code', 'country']])

In [15]:
df_final.head()

,id_ads,manufacturer_by,model,version,km,year,price,seller,endereco,scrapy_datetime,zip_code,country
0,7fd72920-850d-43ba-a452-cbae1a6398da,land rover,DEFENDER 110,SW SE,81000,2015,38000,AUTOMOVILES MARTIN,Contáctanos en: • ES-14440 VILLANUEVA DE CÓRDOBA,2023-10-19 12:22:03,14440,ES
1,840de0a3-82a9-454c-a725-508e211af7de,land rover,DEFENDER 90,td5,147000,1999,24900,AUTOMOVILES MARTIN,Contáctanos en: • ES-14440 VILLANUEVA DE CÓRDOBA,2023-10-19 12:22:03,14440,ES
2,6611c967-ab67-4db5-b28f-37b77fcd332f,land rover,DEFENDER 110,SW,79000,2013,39900,AUTOMOVILES MARTIN,Contáctanos en: • ES-14440 VILLANUEVA DE CÓRDOBA,2023-10-19 12:22:03,14440,ES
3,7484ee53-ebc5-4510-8bf9-032e657837cf,land rover,DEFENDER 110,SW,78000,2012,39500,AUTOMOVILES MARTIN,Contáctanos en: • ES-14440 VILLANUEVA DE CÓRDOBA,2023-10-19 12:22:03,14440,ES
4,08c56b66-b680-4f5c-b63a-33ce190b65eb,land rover,DEFENDER 110,SW SE,52000,2014,42600,AUTOMOVILES MARTIN,Contáctanos en: • ES-14440 VILLANUEVA DE CÓRDOBA,2023-10-19 12:22:03,14440,ES


In [16]:
df_final.dtypes

id_ads             object
manufacturer_by    object
model              object
version            object
km                 object
year                int32
price               int64
seller             object
endereco           object
scrapy_datetime    object
zip_code            int64
country            object
dtype: object

## Save the final dataset


In [17]:
rows = df_final.shape[0]
print(rows)
df_final.to_csv('../dataset/df_final.csv', encoding='utf-8', sep=';')

793
